Practica : entrenar red nbeuronal para clasifiar imagenes de numeros (28x28) 

In [14]:
import torch 

In [14]:
## Un tensor es una manera organizada de almacenar un arreglo de datos (EJ : datos tabulares , imagenes ,audio , texto ) puede ser tanto un vector (1 dimension) como una matriz (2 dimensiones) o mas dimensiones para datos mas complejos (imagenes a color , videos )


## Ejemplo 

Arreglo_datos = [[1,2,3],[4,5,6]]
Tensor_dedatos = torch.tensor(Arreglo_datos)

print(type(Tensor_dedatos))
print(Tensor_dedatos)



<class 'torch.Tensor'>
tensor([[1, 2, 3],
        [4, 5, 6]])


In [16]:
### Otra caracteristica es que estos datos pueden ser procesados tanto por gpu como cpu (torch.cuda.is_available())

Tensor_dedatos.device #.device para saber donde se esta almacenando

##Es posible cambiar el lugar de procesamiento con .to(cpu/cuda)
#Tensor_dedatos.to("cuda").device

device(type='cuda', index=0)

In [17]:
## Al ser un arreglo de datos un Tensor tiene atributos 
Tensor_dedatos.shape

torch.Size([2, 3])

DATA SET 

In [ ]:
## En pytorch existen 2 modulos que nos permitesn  cagar sets de datos 

torch.utils.data.DataLoader # Nos permite crear los bachs para entrenar el modelo (iterar sobre el data set )

torch.utils.data.Dataset # Nos permite leer datos que esten almacenados en X lugar 

In [ ]:
# En caso de estar trabajando con imagenes usaremos 
import torchvision

torchvision.dataset # El cual nos permite cargar el set de datos con imagenes 

Ahora Conociendo estos 2 conceptos (data set y tensor) podemos pasar a la practica 

In [ ]:
# Primero importamos las librerias necesarias 

from torchvision import datasets # descargar el data set (este se encuentra en torch)
from torchvision.transforms import ToTensor # trabajar con tensores 
import matplotlib.pyplot as plt # hacer graficas 

dataset_28x28 = datasets.MNIST (
    
    root='./DATASET/', # donde queremos descargar 
    train=True, # descargar la totalidad de imagenes (cuando trabajamos con un data et para deep learning suele estar dividido en train and validation)
    download=True,
    transform=ToTensor() # Transforma las imagenes a tensores para no tener que hacerlo luego 
    
)


In [2]:
dataset_28x28

Dataset MNIST
    Number of datapoints: 60000
    Root location: ./DATASET/
    Split: Train
    StandardTransform
Transform: ToTensor()

In [ ]:
# observamos el algunos ejemplos del set de datos 

plot = plt.figure(figsize=(8,8))
fila , columna = 3 ,3 

for i in range(1 , columna * fila + 1 ):
    pick = torch.randint(len(dataset_28x28), size=(1,)).item()
    
    imgaen , label = dataset_28x28[pick]
    
    plot.add_subplot(fila,columna,i)
    plt.title(str(label))
    plt.axis('off')
    plt.imshow(imgaen.squeeze() , cmap='gray')
plt.show()

Como mencione antes importamos unas 60 mil imganes pero estan no estan particionadas . 
    siempre que trabajemos en una red nueronal deberemos realizar la partidicion de datos (Entrenamiento , validacion y prueba)
    Nornalmente la particion mas grande es la de entrenamiento donde 80% (segun la cantidad de datos que tengamos)
    luego tenemos la particion de validacion que se usa a la par que la de entrenamiento para validar que tan bien esta respondiendo al entrenamiento 
    y fginalmente con la de prueba probamos el modelo ya entrenado con datos que el modelo nunca vio 

In [9]:
torch.manual_seed(69) #Es recomendable fijar una semilla de aleatoridad para que al volver a correr el codigo tengamos resultados similares 

In [13]:
##Para deividir el set de entrenamiento usaremos usaremos Prueba , validacion , entrenamiento = random_split() dentro de ( pondremos el data set y los procentajes  )


entrenamiento , validacion , prueba = torch.utils.data.random_split( dataset_28x28 , [0.8 , 0.1 , 0.1] )


print(len(entrenamiento))
print(len(validacion))
print(len(prueba))





48000
6000
6000


Ahora con esto podremos empezar con la 'arquitectura' de nuestro modelo 

recordamnos que las redenes neruonales se deividen en 

capas de entrada = cantidad de variables que entren , Por ejemplo en este caso entran 28*28 pixeles de entrada = 784 variables aunque deberemos aplanarla entrada 

capas ocultas = capas donde ocurre el forward propagation

capas de salida = segun la cantidad de categorias o necesidades a predecir (10 digitos = 10 neuronas) y usamos softmax para transformar el valor de salida en una probabilidad 

## Para crear las redes neuronales en torch usamos nn.Module usando POO es decir nuestras redes neuronales van a ser subclases de nn.Module
## Nuestra red neuronal siempre tendra 2 metodos dentro de la subclase 
    INIT = el cual va a definir su arquitectura (cantidad de neuronas , capas etc) 
    forward = el cual va a definir como fluyen los valores por la red neuronal (la conexion entre las neuronas) y como se hace cada prediccion 

In [15]:

from torch import nn 

class Redneuronal(nn.Module): ## al escribir nn.module estamos diciendo que la clase redneuronal es hija de module
    ## estas 2 lineas de abajo siempre iran ya que son estandar de torch para hacer funcionar la red y inicializar el modelo
    def __init__(self):
        super().__init__() 
        #Ahora si modemos crear la arquitectura 
        
        self.aplanar = nn.Flatten() # transforma el tensor de (1x28x28) en un vector de (1 , 784)
        self.red = nn.Sequential( #Esto en torch es una forma de defininir un patron que indica una secuencia de capas aplicadas en el orden dado
            
            nn.Linear(28*28 , 15), ##Primero se define la capa de entrada + con cuantas neuronas se va a conectar (capa oculta)
            nn.ReLU(), ##Aunque arriba hayamos definido la cantidad de neuronas de la capa oculta nos falta definir su funcion de activacion
            nn.Linear(15,10), ## Decimos la capa de salida conectada con las 15 neuronas de la capa oculta 
            nn.Softmax() ##Esto en realidad no es necesario ya que lo unico que hace es transformar los 10 valores de salida en numeros cuya suma sea 1 (probabilidades)
        )
        
        
    def forward(self , x ): # X hace referencia al dato de entrada y la secuancia que va a tomar
        x = self.aplanar(x)
        probabilidades = self.red(x)

        return probabilidades


Ahora con la arquitectura podemos crear una instacian de la clase y moverla a la gpu / cpu 

In [26]:
Modelo = Redneuronal().to('cuda')
Modelo

Redneuronal(
  (aplanar): Flatten(start_dim=1, end_dim=-1)
  (red): Sequential(
    (0): Linear(in_features=784, out_features=15, bias=True)
    (1): ReLU()
    (2): Linear(in_features=15, out_features=10, bias=True)
    (3): Softmax(dim=None)
  )
)

En caso de querer saber la cantidad de pramatros para un modelo podemos uar el metodo parameters()

In [23]:
Parametros = sum(p.numel() for p in Modelo.parameters()) #p.numel() accede a la cantidad de perametros por capa si itera y se suma
print(Parametros)

11935


Con el modelo ya listos podemos pasar a entrenar al modelo pero antes dejaremos los sets de entrenamiento y validacion listos 




In [27]:
from torch.utils.data import DataLoader

# Creamos 2 data loaders uno para el entrenamiento y otro para la validacion (como dije antes estos se usan al mismo tiempo para corroborar que el entrenamiento este yendo bien)
#Shuffle mezcla los datos 

train_loader = DataLoader( 
    dataset=entrenamiento,                          
    batch_size=1000,                      
    shuffle=True                       
                          )

val_loader = DataLoader(
    dataset=validacion,
    batch_size=1000,
    shuffle=True  
)

Ahora pdoemos preparar los hiperparametros para iniciar el entrenamiento 

In [30]:
learning_rate = 0.1 
Epochs = 10 
fun_perdida = nn.CrossEntropyLoss()
optimizador = torch.optim.SGD(Modelo.parameters() , lr = learning_rate)

Y ahora a entrenar 

In [ ]:
def entrenamiento_loop (data_loader , modelo , fun_perdida , optimizador ): 
    #cantidad de datos y cantidad de lotes 
    train_size = len ( data_loader.dataset)
    nlotes = len ( data_loader)
    #pondremos el modelo en modo entrenamiento 
    modelo.train()
    
    #Ahora necesitamos  calcular 2 cosas perdida y exactitud (por cuanto se equivoca y que  tanto)
    perdida , accuracy = 0 ,0 
    
    for nlotes , (x,y) in enumerate(data_loader):
        x , y = x.to("cuda") , y.to('cuda')
        #forward
        probabilidad = modelo(x)
        #backward
        loss = fun_perdida(probabilidad , y)
        loss.backward()
        optimizador.step()
        optimizador.zero_grad()
    
    
        perdida += loss.item()
        accuracy += (probabilidad.argmax(1)==y).type(torch.float).sum().item()
        
        
        if nlotes % 10 : 
            loss ,ndatos = loss.item() , nlotes*train_size
    
        print(f'\tPerdida: {loss.item():>7f} [{ndatos:>5d}/{train_size:>5d}]')
    
    perdida /= nlotes
    accuracy /= train_size
    print(f'\t Exactitud/perdida promedio:')
    print(f'\t\t entrenamiento: {(100*accuracy):>0.1f}% / {perdida:>8f}')